<a href="https://colab.research.google.com/github/440g/painkiller/blob/DP/Data_preprocessing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving news_popular_train.csv to news_popular_train.csv


In [2]:
import pandas as pd

# Load data
df = pd.read_csv('news_popular_train.csv')

In [3]:
#Handling missing values
df = df.drop(columns=['shares'])

#전체 numerical 변수 중앙값으로 일괄 처리 후 일부는 평균값 처리
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop(['id', 'y'])
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

mean_cols = [
    'global_sentiment_polarity', 'global_rate_positive_words',
    'rate_negative_words', 'avg_negative_polarity', 'min_negative_polarity',
    'title_sentiment_polarity', 'title_subjectivity'
]
for col in mean_cols:
    df[col] = df[col].mask(df[col].isna(), df[col].mean())


#categorical 변수 unknown으로 처리
df['data_channel']=df['data_channel'].fillna('unknown')
df['weekday']=df['weekday'].fillna('unknown')

In [4]:
#categorical 변수 One-hot encoding
df = pd.get_dummies(df, columns=['data_channel', 'weekday'], drop_first=True)

In [5]:
#Handling outliers
import numpy as np


'''rate, _polarity 등의 변수 => 0~1 과 같이 일정 구간에서의 값 가짐 => 범위가 정해져있으니 따로 이상치 처리 하지 않아도 될 것 같다고 판단,
이런 종류의 변수를 제외하고 나서 빈도/횟수 값을 갖는 변수들을 대상으로 특히 비대칭이 심한(왜도 극심) 변수들에 대해 IQR을 바탕으로 이상치 처리 '''

#1. IQR 기반 clip
clip_cols = [
    'n_tokens_content', 'num_hrefs', 'num_self_hrefs',
    'kw_avg_avg']
for col in clip_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)

#2. 'num_videos', 'num_imgs' 의 경우, 기사에 포함된 영상/이미지 개수가 너무 큰 경우(200개 등) 비정상적인 것으로 판단=> 95percentile에 해당하는 값을 상한으로 정해서 clip
    df['num_videos'] = df['num_videos'].clip(upper=6)
    df['num_imgs'] = df['num_imgs'].clip(upper=20)

#3. 값 중 51.8%가 -1 => 반 이상 결측값(결측치에 대한 대체값 -1, dummy처리)으로 판단 => 해당 열 제거
df = df.drop(columns=['kw_min_min'])

#4. average_token_length : 본문(내용) 속 단어의 평균 길이 의미, 0의 값을 갖는 것은 비정상적이라고 판단 -> 중앙값으로 대체
df['average_token_length'] = df['average_token_length'].replace(0, np.nan)
df['average_token_length'] = df['average_token_length'].fillna(df['average_token_length'].median())

'''# 5~7 kw_변수-> shares 의미하므로 shares>=0, 음수는 중앙값으로 처리
#6,7 Zeros(value=0)의 비율이 약 40%로 상당하다고 판단=> 결측치에 대한 대체값으로 0이 부여되어 있는 것으로 추측(임의로 0이 부여) => 중앙값으로 다시 대체
'''

#5. 음수(-1)인 값들 -> NaN-> 중앙값으로 처리
df['kw_avg_min'] = df['kw_avg_min'].replace(-1, np.nan)
df['kw_avg_min'] = df['kw_avg_min'].fillna(df['kw_avg_min'].median())
df['kw_avg_min'] = np.log1p(df['kw_avg_min'].clip(lower=0))

#6. kw_min_max, Zeros = 8611 (38.8%) => 결측치의 대체값(영)으로 판단=> 0을 결측치로 간주하여 중앙값으로 다시 대체
df['kw_min_max'] = df['kw_min_max'].replace(0, np.nan)
df['kw_min_max'] = df['kw_min_max'].fillna(df['kw_min_max'].median())

#7. kw_min_avg, 음수값 갖는 경우 존재 & Zeros	8664(39.0%) =>음수값 및 결측치 대체값(영)에 대해 중앙값으로 다시 대체
df['kw_min_avg'] = df['kw_min_avg'].replace(-1, np.nan)
df['kw_min_avg'] = df['kw_min_avg'].replace(0, np.nan)
df['kw_min_avg'] = df['kw_min_avg'].fillna(df['kw_min_avg'].median())

#8. n_tokens_content : 본문 속에 포함된 단어의 개수로 0의 값을 갖는 것은 어색하다고 판단 value=0인 게 594개 정도(2.7%),이에 대한 처리
df = df[df['n_tokens_content'] != 0]

#9. #1~8까지 대략적으로 전처리하고 나서 각 변수에 대한 skew 확인(df.skew()) 결과, 너무 극단적으로 왜도가 큰 변수들이 남아있음을 확인하고 그 변수들에 대해 마지막으로 log1p 스케일링 처리함.
log_transform_cols = ['num_hrefs', 'kw_max_min', 'kw_min_max', 'kw_max_avg','self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess']

for col in log_transform_cols:
  df[col] = np.log1p(df[col])

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# numerical inputs
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop(['id', 'y'])

# scaling
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

X = df.drop(columns=['id', 'y'])
y = df['y']

In [7]:
#Data split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)